In [1]:
import pandas as pd
import re
import csv
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

tweets = pd.read_csv('../tweets/tweets.csv', header=0)
print(tweets.isna().sum())
print()

C:\Users\jmelt\AppData\Local\Programs\Python\Python36-32\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
  return f(*args, **kwds)
C:\Users\jmelt\AppData\Local\Programs\Python\Python36-32\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 12 from C header, got 20 from PyObject
  return f(*args, **kwds)


user_id                    8065
user_key                      0
created_at                   21
created_str                  21
retweet_count            145399
retweeted                145399
favorite_count           145399
text                         21
tweet_id                   2314
source                   145398
hashtags                      0
expanded_urls                 0
posted                        0
mentions                      0
retweeted_status_id      163802
in_reply_to_status_id    202892
dtype: int64



In [2]:
tweets.drop(['user_id', 'created_at', 'retweet_count', 'retweeted',
             'favorite_count', 'source', 'expanded_urls', 'posted',
             'retweeted_status_id', 'in_reply_to_status_id'],
            axis=1, inplace=True)

# Remove rows with NaN text field
tweets.dropna(subset=['text'], inplace=True)
tweets.reset_index(drop=True, inplace=True)

tweets.rename(columns={'created_str': 'date'}, inplace=True)

tweets.info()
print()
print('Unique account keys:', len(tweets.user_key.unique()))
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203430 entries, 0 to 203429
Data columns (total 6 columns):
user_key    203430 non-null object
date        203430 non-null object
text        203430 non-null object
tweet_id    201116 non-null float64
hashtags    203430 non-null object
mentions    203430 non-null object
dtypes: float64(1), object(5)
memory usage: 5.4+ MB

Unique account keys: 453



In [3]:
# Find hashtags in tweets
print('Finding hashtags in tweets')
tweets['hashtags'] = tweets['text'].apply(lambda text: re.findall('#(\w+)', text))

Finding hashtags in tweets


In [4]:
def clean_text(text):
    from bs4 import BeautifulSoup

    # Remove URLs
    www_exp = r'www.[^ ]+'
    http_exp = r'http?s?[^\s]+'
    clean = re.sub('|'.join((www_exp, http_exp)), '', text)

    # Remove HTML encoded text (ampersand)
    soup = BeautifulSoup(clean, 'lxml')
    clean = soup.get_text()
    try:
        clean = clean.encode().decode().replace(u'\ufffd', '?')
    except UnicodeEncodeError or UnicodeDecodeError:
        clean = clean

    # Split 'not' contractions
    contraction_dict = {'can\'t': 'can not', 'won\'t': 'will not',
                        'isn\'t': 'is not', 'aren\'t': 'are not',
                        'wasn\'t': 'was not', 'weren\'t': 'were not',
                        'haven\'t': 'have not', 'hasn\'t': 'has not',
                        'wouldn\'t': 'would not', 'don\'t': 'do not',
                        'doesn\'t': 'does not', 'didn\'t': 'did not',
                        'couldn\'t': 'could not', 'shouldn\'t': 'should not',
                        'mightn\'t': 'might not', 'mustn\'t': 'must not',
                        'had\'t': 'had not'}
    contraction_exp = re.compile(r'\b(' + '|'.join(contraction_dict.keys()) + r')\b')
    clean = contraction_exp.sub(lambda x: contraction_dict[x.group()], clean)

    # Remove @ mentions and hashtags
    # at_exp = r'@[A-Za-z0-9_]+'
    # hashtag_exp = r'#[A-Za-z0-9_]+'
    # clean = re.sub('|'.join((at_exp, hashtag_exp)), '', clean)

    # Remove non-letter chars, 'RT'/'MT' from retweets, enclitics from split contractions
    clean = re.sub('[^a-zA-Z0-9]', ' ', clean)
    tails = [r'\bRT\b', r'\bMT\b', r'\bve\b', r'\bre\b', r'\bll\b']
    clean = re.sub('|'.join(tails), '', clean)
    # Convert to lower case
    clean = clean.lower()

    return ' '.join([word for word in clean.split(' ') if word is not ''])


# Apply preprocessing to tweet text
print('Cleaning tweet text')
tweets['clean_text'] = tweets['text'].apply(clean_text)
tweets.dropna(subset=['clean_text'], inplace=True)
tweets.reset_index(drop=True, inplace=True)

Cleaning tweet text


C:\Users\jmelt\AppData\Roaming\Python\Python36\site-packages\bs4\__init__.py:273: UserWarning: "b'... '" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\jmelt\AppData\Roaming\Python\Python36\site-packages\bs4\__init__.py:273: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [5]:
tweet_text = list(tweets.clean_text)

# Tokenize tweets and remove stop words
stop_words = set(stopwords.words('english'))
my_stops = ['go', 'be', 'also', 'get', 'do', 'thing', 'use', 'let', 'would', 'say', 'could', 'yet']
stop_words.update(my_stops)

print('Tokenizing and removing stop words')
tweet_text = [[word for word in word_tokenize(tweet) if word not in stop_words and len(word) > 2]
              for tweet in tweet_text]

nlp = spacy.load('en', disable=['parser', 'ner'])


def lemmatization(texts, allowed_postags=('NOUN', 'ADJ', 'VERB', 'ADV')):
    lemmas = []
    for i, tweet in enumerate(texts):
        if i % 10000 == 0 or i == len(tweets)-1:
            print('Lemmatized {} tweets'.format(i))
        lemmas.append([token.lemma_ for token in nlp(' '.join(tweet))
                       if token.pos_ in allowed_postags and token.lemma_ not in stop_words])

    return lemmas


tweet_lemmas = lemmatization(tweet_text)
tweets['lemmas'] = [' '.join(words) for words in tweet_lemmas]
tweets.dropna(subset=['lemmas'], inplace=True)
tweets.reset_index(drop=True, inplace=True)

print()
tweets.info()

tweets.to_csv('../tweets/tweets_clean.csv', index=False, quoting=csv.QUOTE_ALL)

Tokenizing and removing stop words


C:\Users\jmelt\AppData\Local\Programs\Python\Python36-32\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
  return f(*args, **kwds)
C:\Users\jmelt\AppData\Local\Programs\Python\Python36-32\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 12 from C header, got 20 from PyObject
  return f(*args, **kwds)


Lemmatized 0 tweets
Lemmatized 10000 tweets
Lemmatized 20000 tweets
Lemmatized 30000 tweets
Lemmatized 40000 tweets
Lemmatized 50000 tweets
Lemmatized 60000 tweets
Lemmatized 70000 tweets
Lemmatized 80000 tweets
Lemmatized 90000 tweets
Lemmatized 100000 tweets
Lemmatized 110000 tweets
Lemmatized 120000 tweets
Lemmatized 130000 tweets
Lemmatized 140000 tweets
Lemmatized 150000 tweets
Lemmatized 160000 tweets
Lemmatized 170000 tweets
Lemmatized 180000 tweets
Lemmatized 190000 tweets
Lemmatized 200000 tweets
Lemmatized 203429 tweets

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203430 entries, 0 to 203429
Data columns (total 8 columns):
user_key      203430 non-null object
date          203430 non-null object
text          203430 non-null object
tweet_id      201116 non-null float64
hashtags      203430 non-null object
mentions      203430 non-null object
clean_text    203430 non-null object
lemmas        203430 non-null object
dtypes: float64(1), object(7)
memory usage: 7.0+ MB
